# Desafio 3

Neste desafio, iremos praticar nossos conhecimentos sobre distribuições de probabilidade. Para isso,
dividiremos este desafio em duas partes:
    
1. A primeira parte contará com 3 questões sobre um *data set* artificial com dados de uma amostra normal e
    uma binomial.
2. A segunda parte será sobre a análise da distribuição de uma variável do _data set_ [Pulsar Star](https://archive.ics.uci.edu/ml/datasets/HTRU2), contendo 2 questões.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF

In [2]:
%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

## Parte 1

### _Setup_ da parte 1

In [3]:
np.random.seed(42)
    
dataframe = pd.DataFrame({"normal": sct.norm.rvs(20, 4, size=10000),
                     "binomial": sct.binom.rvs(100, 0.2, size=10000)})

## Inicie sua análise a partir da parte 1 a partir daqui

In [4]:
## Inicie sua análise a partir da parte 1 a partir daqui
dataframe.describe()

,normal,binomial
count,10000.000000,10000.000000
mean,19.991456,20.097000
std,4.013850,4.041191
min,4.310399,6.000000
25%,17.309638,17.000000
50%,19.989620,20.000000
75%,22.684324,23.000000
max,35.704951,36.000000


## Questão 1

Qual a diferença entre os quartis (Q1, Q2 e Q3) das variáveis `normal` e `binomial` de `dataframe`? Responda como uma tupla de três elementos arredondados para três casas decimais.

Em outra palavras, sejam `q1_norm`, `q2_norm` e `q3_norm` os quantis da variável `normal` e `q1_binom`, `q2_binom` e `q3_binom` os quantis da variável `binom`, qual a diferença `(q1_norm - q1 binom, q2_norm - q2_binom, q3_norm - q3_binom)`?

In [5]:
def q1():
    
    q_norm = dataframe['normal'].quantile((0.25, 0.5, 0.75))
    q_binom = dataframe['binomial'].quantile((0.25, 0.5, 0.75))

    dif = q_norm - q_binom
    
    return tuple (round(dif, 3))
    pass

q1()

(0.31, -0.01, -0.316)

Para refletir:

* Você esperava valores dessa magnitude?

* Você é capaz de explicar como distribuições aparentemente tão diferentes (discreta e contínua, por exemplo) conseguem dar esses valores?

## Questão 2

Considere o intervalo $[\bar{x} - s, \bar{x} + s]$, onde $\bar{x}$ é a média amostral e $s$ é o desvio padrão. Qual a probabilidade nesse intervalo, calculada pela função de distribuição acumulada empírica (CDF empírica) da variável `normal`? Responda como uma único escalar arredondado para três casas decimais.

In [6]:
def q2():
    
    #Calculando a média e o desvio padrão
    x = dataframe.normal.mean()
    s = dataframe.normal.std()

    #Usando CDF para a probalidade do 1º valor do intervaldo
    cdf = sct.norm.cdf(x-s, loc=x, scale=s)
    
    #Usando SF para a probalidade do 2º valor do intervaldo
    sf = sct.norm.sf(x+s, loc=x, scale=s)

    #Subtraindo a soma das probabilidades da área 1
    prob = 1 - (cdf + sf)

    return round(prob, 3)
    pass

q2()

0.683

Para refletir:

* Esse valor se aproxima do esperado teórico?
* Experimente também para os intervalos $[\bar{x} - 2s, \bar{x} + 2s]$ e $[\bar{x} - 3s, \bar{x} + 3s]$.

## Questão 3

Qual é a diferença entre as médias e as variâncias das variáveis `binomial` e `normal`? Responda como uma tupla de dois elementos arredondados para três casas decimais.

Em outras palavras, sejam `m_binom` e `v_binom` a média e a variância da variável `binomial`, e `m_norm` e `v_norm` a média e a variância da variável `normal`. Quais as diferenças `(m_binom - m_norm, v_binom - v_norm)`?

In [7]:
def q3():
    
    #Calculando a média e variância da variável binomial
    m_binom = dataframe.binomial.mean()
    v_binom = dataframe.binomial.var()
    
    #Calculando a média e variância da variável normal
    m_norm = dataframe.normal.mean()
    v_norm = dataframe.normal.var()
    
    #Calculando a diferença
    return (round(m_binom - m_norm, 3), round(v_binom - v_norm, 3))
    pass

q3()

(0.106, 0.22)

Para refletir:

* Você esperava valore dessa magnitude?
* Qual o efeito de aumentar ou diminuir $n$ (atualmente 100) na distribuição da variável `binomial`?

## Parte 2

### _Setup_ da parte 2

In [8]:
stars = pd.read_csv("pulsar_stars.csv")

stars.rename({old_name: new_name
              for (old_name, new_name)
              in zip(stars.columns,
                     ["mean_profile", "sd_profile", "kurt_profile", "skew_profile", "mean_curve", "sd_curve", "kurt_curve", "skew_curve", "target"])
             },
             axis=1, inplace=True)

stars.loc[:, "target"] = stars.target.astype(bool)

## Inicie sua análise da parte 2 a partir daqui

## Questão 4

Considerando a variável `mean_profile` de `stars`:

1. Filtre apenas os valores de `mean_profile` onde `target == 0` (ou seja, onde a estrela não é um pulsar).
2. Padronize a variável `mean_profile` filtrada anteriormente para ter média 0 e variância 1.

Chamaremos a variável resultante de `false_pulsar_mean_profile_standardized`.

Encontre os quantis teóricos para uma distribuição normal de média 0 e variância 1 para 0.80, 0.90 e 0.95 através da função `norm.ppf()` disponível em `scipy.stats`.

Quais as probabilidade associadas a esses quantis utilizando a CDF empírica da variável `false_pulsar_mean_profile_standardized`? Responda como uma tupla de três elementos arredondados para três casas decimais.

In [9]:
def q4():

    #Filtrando valores onde target == 0
    fpms = stars[stars.target == 0]

    #Padronizando variável mean_profile
    false_pulsar_mean_profile_standardized = (fpms['mean_profile'] - fpms['mean_profile'].mean()) /fpms['mean_profile'].std()  

    #Encontrando os quantis teóricos usando norm.ppf e ECDF
    a = sct.norm.ppf(0.80)
    b = sct.norm.ppf(0.90)
    c = sct.norm.ppf(0.95)

    ecdf = ECDF(false_pulsar_mean_profile_standardized)
    ecdf_a = round(ecdf(a),3)
    ecdf_b = round(ecdf(b),3)
    ecdf_c = round(ecdf(c),3)

    return (ecdf_a,ecdf_b,ecdf_c) 
    pass

q4()

(0.806, 0.911, 0.959)

Para refletir:

* Os valores encontrados fazem sentido?
* O que isso pode dizer sobre a distribuição da variável `false_pulsar_mean_profile_standardized`?

## Questão 5

Qual a diferença entre os quantis Q1, Q2 e Q3 de `false_pulsar_mean_profile_standardized` e os mesmos quantis teóricos de uma distribuição normal de média 0 e variância 1? Responda como uma tupla de três elementos arredondados para três casas decimais.

In [10]:
def q5():
    
    #Filtrando valores onde target == 0  e padronizando variável mean_profile (idem acima)
    fpms = stars[stars.target == 0]
    false_pulsar_mean_profile_standardized = (fpms['mean_profile'] - fpms['mean_profile'].mean()) /fpms['mean_profile'].std() 
    
    #Separando quantis de distribuição normal e da false_pulsar_mean_profile_standardized
    q_normal =  (sct.norm.ppf((0.25, 0.5, 0.75)))
    q_fpms = (false_pulsar_mean_profile_standardized.quantile((0.25, 0.5, 0.75)))
    
    #Calculando a diferença
    return tuple (round(q_fpms - q_normal, 3))
    pass

q5()

(0.027, 0.04, -0.004)

Para refletir:

* Os valores encontrados fazem sentido?
* O que isso pode dizer sobre a distribuição da variável `false_pulsar_mean_profile_standardized`?
* Curiosidade: alguns testes de hipóteses sobre normalidade dos dados utilizam essa mesma abordagem.